In [71]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import os
import re

In [72]:
idx = [(1, 0)]
for z, c in idx:
    data_set = pickle.load(open('goal_set.p', 'rb'))
    data = data_set['train'] + data_set['test'] + data_set['validate']
    dis_s = {}
    symptoms = {}
    for xy in data:
        disease = xy['disease_tag']
        dis_s.setdefault(disease, {'index':len(dis_s), 'symptom':dict()})
        temp2 = xy['goal']['implicit_inform_slots']
        temp2.update(xy['goal']['explicit_inform_slots'])
        for x, y in temp2.items():
            symptoms.setdefault(x, len(symptoms))
            dis_s[disease]['symptom'].setdefault(x, 0)
            dis_s[disease]['symptom'][x] += 1
    #for x in dis_s:
        #frequency = 0
        #for y in dis_s[x]['symptom']:
            #frequency += dis_s[x]['symptom'][y]
        #for y in dis_s[x]['symptom']:
            #dis_s[x]['symptom'][y] /= frequency
    symptoms['Total Symptom'] = len(symptoms)
    pickle.dump(obj = dis_s, file = open('out1.p', 'wb'), protocol = 2)
    pickle.dump(obj = symptoms, file = open('out2.p', 'wb'), protocol = 2)

TypeError: 'int' object is not callable

In [68]:
idx = [(1, 0)]
#idx = [(1, 0)]
for z, c in idx:
    print(z)
    temp_dict = pickle.load(open('out2.p', 'rb'))
    temp_data = pickle.load(open('out1.p', 'rb'))
    length = 0

    dict = {}
    dict1 = {}
    dict2 = {}
    dict3 = {}

    for xy in temp_dict:
        dict[length] = xy
        dict1[xy] = length
        length += 1

    length = 0
    for xy in temp_data:
        dict2[length] = xy
        dict3[xy] = length
        length += 1

    data_set = pickle.load(open(r'label' + str(z) + '\goal_set.p', 'rb'))
    data = data_set['train'] + data_set['test'] + data_set['validate']
    count1 = 0
    
    for xy in data:
        count1 += 1

    X = np.zeros(shape=(count1, temp_dict["Total Symptom"]), dtype=np.uint8)
    y = np.zeros(shape=(1, count1), dtype=np.uint8)
    y = y.flatten()
    count1 = 0
    
    
    for xy in data:
        for xx in xy['goal']['explicit_inform_slots']:
            X[count1][dict1[xx]] = 1
        for xx in xy['goal']['implicit_inform_slots']:
            X[count1][dict1[xx]] = 1
        y[count1] = dict3[xy['disease_tag']]
        count1 += 1
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    #a = model.feature_importances_
    tot = int(temp_dict["Total Symptom"])
    tot = int((90*tot + 99)/100)
    #bestfeatures = SelectKBest(score_func=chi2, k = tot)
    bestfeatures = ExtraTreesClassifier(n_estimators = tot)
    fit = bestfeatures.fit(X_train, y_train)
    #X = bestfeatures.transform(X)
    #dfscores = fit.scores_
    dfscores = bestfeatures.feature_importances_
    print(dfscores)
    l = []
    timer = 0
    for i in dfscores:
        l.append((i, dict[timer]))
        timer += 1
    l.sort()
    res = {}
    for i, j in l:
        res[j] = float(i)
    pickle.dump(obj = res, file = open('out3.p', 'wb'), protocol = 2)
    l.reverse()
    r = []
    res = {}
    for i in range (0, tot):
        r.append(l[i])
    res = {}
    timer = 0
    for i, j in r:
        res[j] = timer
        timer += 1
    pickle.dump(obj = res, file = open('out4.p', 'wb'), protocol = 2)

1
[9.81186050e-03 1.32311896e-02 1.09380612e-02 1.31823783e-02
 9.24138085e-02 6.78587792e-02 1.89684387e-02 3.58696074e-02
 4.91679726e-02 2.40804195e-02 2.85074872e-02 1.16862415e-02
 1.16510839e-02 1.41765510e-02 1.26412395e-02 2.70497180e-02
 3.72907764e-02 1.12576138e-02 1.08735364e-02 1.11550835e-02
 1.58136848e-02 9.26856420e-03 6.05198699e-03 1.23509552e-02
 5.53774166e-02 7.70044223e-02 1.30161866e-02 3.01595817e-02
 1.85382956e-02 1.06749658e-02 1.08746928e-02 1.04285645e-02
 1.18562084e-02 2.49513921e-02 4.18446989e-03 7.40157324e-03
 1.72939296e-04 3.62140419e-02 5.19554170e-05 7.55883284e-04
 2.99721366e-02 1.45023313e-02 1.03632092e-02 1.21102724e-04
 3.27194794e-03 8.78395191e-03 8.90162807e-03 1.30072283e-04
 2.45859741e-03 1.10666977e-02 9.05177114e-03 3.01064513e-04
 1.02828077e-03 2.93100624e-03 3.85336121e-03 7.30790758e-04
 1.22802870e-03 6.02604034e-03 2.51509571e-04 1.04305745e-02
 6.96680445e-03 2.45972667e-04 1.43683904e-04 1.08454890e-04
 1.71353725e-04]
2
[4.

In [69]:
idx = [(1, 0)]
x = []
for z, c in idx:
    temp_dict = pickle.load(open('out2.p', 'rb'))
    temp_data = pickle.load(open('out1.p', 'rb'))
    length = 0

    dict0 = {}
    dict1 = {}
    dict2 = {}
    dict3 = {}

    for xy in temp_dict:
        dict0[length] = xy
        dict1[xy] = length
        length += 1
    
    length = 0
    for xy in temp_data:
        dict2[length] = xy
        dict3[xy] = length
        length += 1
    #data_set = pickle.load(open(r'label' + str(z) + '\goal_set.p', 'rb'))
    #data = data_set['train'] + data_set['test'] + data_set['validate']
    X = np.zeros(shape=(length, temp_dict["Total Symptom"]), dtype=np.uint8)
    
    for xy in temp_data:
        for xx in temp_data[xy]['symptom']:
            X[dict3[xy]][dict1[xx]] = 1
    
    l = {}
    len = 0
    
    for xx in range(0, temp_dict["Total Symptom"]):
        cur = 0
        r = {}
        for yy in range(0, length):
            if X[yy][xx] == 1:
                r[dict2[yy]] = cur
                cur += 1
        l[dict0[xx]] = r
    
    pickle.dump(obj = l, file = open('out5.p', 'wb'), protocol = 2)

## Disease Classifier

In [70]:
idx = [(1, 0)]
x = []
for z, c in idx:
    temp_dict = pickle.load(open('out4.p', 'rb'))
    temp_data = pickle.load(open('out1.p', 'rb'))
    
    data_set = pickle.load(open('goal_set.p', 'rb'))
    data = data_set['train'] + data_set['test'] + data_set['validate']
    
    dict_sym_i ={}
    dict_i_sym = {}
    total_sym = 0
    for xy in temp_dict:
        dict_sym_i[xy] = total_sym
        dict_i_sym[total_sym] = xy
        total_sym += 1
    pickle.dump(obj = dict_sym_i, file = open('dict_sym_i.p', 'wb'), protocol = 2)
    pickle.dump(obj = dict_i_sym, file = open('dict_i_sym.p', 'wb'), protocol = 2)
    
    dict_dis_i = {}
    dict_i_dis = {}
    total_dis = 0
    for xy in temp_data:
        dict_dis_i[xy] = total_dis
        dict_i_dis[total_dis] = xy
        total_dis +=1
    pickle.dump(obj = dict_dis_i, file = open('dict_dis_i.p', 'wb'), protocol = 2)
    pickle.dump(obj = dict_i_dis, file = open('dict_i_dis.p', 'wb'), protocol = 2)
    
    total_dia = 0
    for xy in data:
        total_dia += 1
    
    print(total_sym)
    X = np.zeros(shape=(total_dia, total_sym), dtype=np.uint8)
    y = np.zeros(shape=(1, total_dia), dtype=np.uint8)
    y = y.flatten()
    total_dia = 0
    
    for xy in data:
        for xx in xy['goal']['explicit_inform_slots']:
            if xx in dict_sym_i:
                X[total_dia][dict_sym_i[xx]] = 1
        for xx in xy['goal']['implicit_inform_slots']:
            if xx in dict_sym_i:
                X[total_dia][dict_sym_i[xx]] = 1
        y[total_dia] = dict_dis_i[xy['disease_tag']]
        total_dia += 1
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    pickle.dump(classifier, open('model.sav', 'wb'))

59
0.855
81
0.984
62
0.863
53
0.927
42
0.784
46
0.797
56
0.952
63
0.957
66
0.874


In [57]:
list = []
with open('test.txt','r') as file:        
    for line in file: 
        list.append(line)
idx = [(1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8)]
for z, c in idx:
    dict_sym_i = pickle.load(open(r'label' + str(z) + '\dict_sym_i.p', 'rb'))
    dict_i_dis = pickle.load(open(r'label' + str(z) + '\dict_i_dis.p', 'rb'))
    total_sym = 0
    for xy in dict_sym_i:
        total_sym += 1
    X = np.zeros(shape=(1, total_sym), dtype=np.uint8)
    for xy in list:
        if xy in dict_sym_i:
            X[0][dict_sym_i[xy]] = 1
    classifier = pickle.load(open(r'label' + str(z) + '\model.sav', 'rb'))
    y_pred = classifier.predict(X)
    print(dict_i_dis[y_pred[0]])

Chlamydia
Cushing syndrome
Adjustment reaction
Extrapyramidal effect of drugs
Acute glaucoma
Diaper rash
Flat feet
Endometriosis
Corneal abrasion


In [ ]:
idx = [(1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8)]
for z, c in idx:
    dict_sym_i = pickle.load(open(r'label' + str(z) + '\dict_sym_i.p', 'rb'))
    dict_i_dis = pickle.load(open(r'label' + str(z) + '\dict_i_dis.p', 'rb'))
    data_set = pickle.load(open(r'label' + str(z) + '\goal_set.p', 'rb'))
    data = data_set['train'] + data_set['test'] + data_set['validate']
    count1 = 0
    for xy in data:
        count1 += 1
    total_sym = 0
    for xy in dict_sym_i:
        total_sym += 1
    X = np.zeros(shape=(count1, total_sym), dtype=np.uint8)
    